In [1]:
import tiktoken
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

from gpt_model import GPTModel
from clean_gutenberg_text import clean_gutenberg_text
from train_sae import train_sae

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

print(f"Using {device} device.")

Using mps device.


In [3]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,
    "context_length": 256,
    "emb_dim": 768,
    "n_heads": 12,
    "n_layers": 12,
    "drop_rate": 0.2,
    "qkv_bias": False,
    "device": device,
}

In [4]:
model = GPTModel(GPT_CONFIG_124M)
checkpoint = torch.load("model_and_optimizer_5.pth", weights_only=True)

model.load_state_dict(checkpoint["model_state_dict"])
model.to(device)
model.eval();

In [5]:
tokenizer = tiktoken.get_encoding("gpt2")

In [6]:
import re

def load_and_clean_text(file_path):
    """
    Loads a text file and splits it into sentences while cleaning the text.
    
    Args:
    - file_path (str): Path to the text file.
    
    Returns:
    - list: A list of cleaned sentences from the book.
    """
    
    text = clean_gutenberg_text(file_path)

    # Split text into sentences (simple heuristic using punctuation)
    sentences = re.split(r"(?<=[.!?])\s+", text)

    # Remove very short or long sentences
    sentences = [s.strip() for s in sentences if 5 < len(s.split()) < 50]

    return sentences

In [7]:
directory="original_texts/"
dataset = []

for filename in os.listdir(directory):
    if filename.endswith(".txt"):
        sentences = load_and_clean_text(os.path.join(directory, filename))
        dataset += sentences

In [8]:
def text_to_token_ids(text, tokenizer):
    encoded = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
    encoded_tensor = torch.tensor(encoded).unsqueeze(0) # add batch dimension
    return encoded_tensor

def get_token_embeddings(text, model, tokenizer, layers=[6, 12]):
    """
    Extracts token embeddings from specified transformer layers.

    Args:
    - text (str): Input text.
    - model: Custom GPT model.
    - tokenizer: tiktoken encoding object.
    - layers (list): Transformer layers to extract embeddings from.

    Returns:
    - dict: Layer-wise token embeddings {layer_number: embeddings}
    """

    input_ids = text_to_token_ids(text, tokenizer).to(device)

    with torch.no_grad():
        _, hidden_states = model(input_ids, output_hidden_states=True)

    embeddings = {}
    for layer in layers:
        if layer - 1 < len(hidden_states):
            embeddings[layer] = hidden_states[layer - 1].squeeze(0).cpu().numpy()
        else:
            print(f"⚠️ Warning: Layer {layer} is out of range (max index {len(hidden_states) - 1})")

    return embeddings

In [ ]:
layer6_embeddings = []
layer12_embeddings = []

for sentence in dataset:
    embeddings = get_token_embeddings(sentence, model, tokenizer)
    layer6_embeddings.append(embeddings[6])
    layer12_embeddings.append(embeddings[12])

# Convert to NumPy and flatten tokens into dataset
layer6_embeddings = np.vstack(layer6_embeddings)
layer12_embeddings = np.vstack(layer12_embeddings)

os.makedirs("sae_data", exist_ok=True)
np.save("sae_data/layer6_embeddings.npy", layer6_embeddings)
np.save("sae_data/layer12_embeddings.npy", layer12_embeddings)

print("Saved token embeddings:")
print(f"Layer 6: {layer6_embeddings.shape}")
print(f"Layer 12: {layer12_embeddings.shape}")

In [9]:
layer6_embeddings = np.load("sae_data/layer6_embeddings.npy")
layer12_embeddings = np.load("sae_data/layer12_embeddings.npy")

In [11]:
import optuna
import torch
import numpy as np
from torch.utils.data import DataLoader, TensorDataset, random_split
import torch.optim as optim
import torch.nn as nn
from sparse_auto_encoder import SparseAutoencoder

def objective(trial, device="cpu", embeddings_path="training_embeddings.npy"):
    # Hyperparameter search space
    print(50*"=")
    print(f"Trial number {trial.number + 1}")
    print(50*"=")
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])
    lr = trial.suggest_loguniform("lr", 1e-4, 1e-2)
    hidden_dim = trial.suggest_int("hidden_dim", 64, 512, step=64)
    weight_decay = trial.suggest_loguniform("weight_decay", 1e-6, 1e-3)

    # Load your embeddings
    embeddings = np.load(embeddings_path)  # Replace with actual file
    embeddings = torch.tensor(embeddings, dtype=torch.float32).to(device)

    input_dim = embeddings.shape[1]
    sae = SparseAutoencoder(input_dim=input_dim, hidden_dim=hidden_dim).to(device)

    optimizer = optim.AdamW(sae.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = nn.MSELoss()

    dataset = TensorDataset(embeddings)
    train_size = int(0.9 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

    best_val_loss = float("inf")
    patience, early_stop_counter = 10, 0

    for epoch in range(30):
        sae.train()
        train_loss = 0.0
        for batch in train_loader:
            inputs = batch[0].to(device)
            optimizer.zero_grad()
            outputs, encoded = sae(inputs)
            loss = criterion(outputs, inputs)
            sparsity_loss = torch.norm(encoded, p=1) * 1e-4
            total_loss = loss + sparsity_loss
            total_loss.backward()
            optimizer.step()
            train_loss += total_loss.item()

        sae.eval()
        val_loss = 0.0
        with torch.no_grad():
            for batch in val_loader:
                inputs = batch[0].to(device)
                outputs, encoded = sae(inputs)
                loss = criterion(outputs, inputs)
                sparsity_loss = torch.norm(encoded, p=1) * 1e-4
                total_loss = loss + sparsity_loss
                val_loss += total_loss.item()

        train_loss /= len(train_loader)
        val_loss /= len(val_loader)

        scheduler.step(val_loss)
        print(f"Epoch {epoch}: Train loss {train_loss:.4f}, Val loss {val_loss:.4f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            early_stop_counter = 0
        else:
            early_stop_counter += 1

        if early_stop_counter >= patience:
            break

    return best_val_loss 

In [12]:
import time

start_time = time.time()

# Run hyperparameter tuning
study = optuna.create_study(direction="minimize")
study.optimize(lambda trial: objective(trial, device, embeddings_path="sae_data/layer6_embeddings.npy"), n_trials=15)

# Print best hyperparameters
print("Best hyperparameters:", study.best_params)

end_time = time.time()
execution_time_minutes = (end_time - start_time) / 60
print(f"Search completed in {execution_time_minutes:.2f} minutes.")

[I 2025-03-07 05:49:50,242] A new study created in memory with name: no-name-debab08a-1ae8-4354-b1d7-2e8a93fcaf53
/var/folders/c1/qdg0q3b92ys5hzf9t6h4xvf40000gn/T/ipykernel_78150/160072725.py:15: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform("lr", 1e-4, 1e-2)
/var/folders/c1/qdg0q3b92ys5hzf9t6h4xvf40000gn/T/ipykernel_78150/160072725.py:17: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform("weight_decay", 1e-6, 1e-3)


Trial number 1
Epoch 0: Train loss 3.5013, Val loss 2.7664
Epoch 1: Train loss 2.6397, Val loss 2.5404
Epoch 2: Train loss 2.5101, Val loss 2.4382
Epoch 3: Train loss 2.4494, Val loss 2.3934
Epoch 4: Train loss 2.4061, Val loss 2.3609
Epoch 5: Train loss 2.3730, Val loss 2.3305
Epoch 6: Train loss 2.3432, Val loss 2.3005
Epoch 7: Train loss 2.3171, Val loss 2.2783
Epoch 8: Train loss 2.2942, Val loss 2.2555
Epoch 9: Train loss 2.2737, Val loss 2.2374
Epoch 10: Train loss 2.2572, Val loss 2.2252
Epoch 11: Train loss 2.2438, Val loss 2.2149
Epoch 12: Train loss 2.2321, Val loss 2.2054
Epoch 13: Train loss 2.2236, Val loss 2.2009
Epoch 14: Train loss 2.2164, Val loss 2.1995
Epoch 15: Train loss 2.2110, Val loss 2.1948
Epoch 16: Train loss 2.2063, Val loss 2.1908
Epoch 17: Train loss 2.2020, Val loss 2.1906
Epoch 18: Train loss 2.1982, Val loss 2.1864
Epoch 19: Train loss 2.1944, Val loss 2.1859
Epoch 20: Train loss 2.1912, Val loss 2.1838
Epoch 21: Train loss 2.1882, Val loss 2.1842
Epoch

[I 2025-03-07 06:02:12,080] Trial 0 finished with value: 2.170551270788366 and parameters: {'batch_size': 128, 'lr': 0.0002688498549528964, 'hidden_dim': 320, 'weight_decay': 0.00024172216807324387}. Best is trial 0 with value: 2.170551270788366.


Epoch 29: Train loss 2.1711, Val loss 2.1706
Trial number 2


/var/folders/c1/qdg0q3b92ys5hzf9t6h4xvf40000gn/T/ipykernel_78150/160072725.py:15: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform("lr", 1e-4, 1e-2)
/var/folders/c1/qdg0q3b92ys5hzf9t6h4xvf40000gn/T/ipykernel_78150/160072725.py:17: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform("weight_decay", 1e-6, 1e-3)


...

Epoch 11: Train loss 2.1431, Val loss 2.2010
Trial number 14
Epoch 0: Train loss 2.5095, Val loss 2.2279
Epoch 1: Train loss 2.2277, Val loss 2.1206
Epoch 2: Train loss 2.1678, Val loss 2.1205
Epoch 3: Train loss 2.1543, Val loss 2.1307
Epoch 4: Train loss 2.1478, Val loss 2.1580
Epoch 5: Train loss 2.1434, Val loss 2.1555
Epoch 6: Train loss 2.1400, Val loss 2.1728
Epoch 7: Train loss 2.1374, Val loss 2.1752
Epoch 8: Train loss 2.1244, Val loss 2.0986
Epoch 9: Train loss 2.1213, Val loss 2.1100
Epoch 10: Train loss 2.1196, Val loss 2.1081
Epoch 11: Train loss 2.1185, Val loss 2.1205
Epoch 12: Train loss 2.1173, Val loss 2.1234
Epoch 13: Train loss 2.1164, Val loss 2.1237
Epoch 14: Train loss 2.1156, Val loss 2.1255
Epoch 15: Train loss 2.1085, Val loss 2.0756
Epoch 16: Train loss 2.1080, Val loss 2.0798
Epoch 17: Train loss 2.1071, Val loss 2.0785
Epoch 18: Train loss 2.1066, Val loss 2.0757
Epoch 19: Train loss 2.1058, Val loss 2.0778
Epoch 20: Train loss 2.1053, Val loss 2.0760
Epoc

[I 2025-03-07 11:26:55,833] Trial 13 finished with value: 2.05181546670681 and parameters: {'batch_size': 32, 'lr': 0.0005587614855267997, 'hidden_dim': 512, 'weight_decay': 6.056437420027853e-06}. Best is trial 13 with value: 2.05181546670681.


Epoch 29: Train loss 2.0975, Val loss 2.0518
Trial number 15
Epoch 0: Train loss 2.8744, Val loss 2.4504
Epoch 1: Train loss 2.4838, Val loss 2.3448
Epoch 2: Train loss 2.4184, Val loss 2.2957
Epoch 3: Train loss 2.3694, Val loss 2.2535
Epoch 4: Train loss 2.3175, Val loss 2.1969
Epoch 5: Train loss 2.2606, Val loss 2.1474
Epoch 6: Train loss 2.2106, Val loss 2.1180
Epoch 7: Train loss 2.1856, Val loss 2.1069
Epoch 8: Train loss 2.1742, Val loss 2.1009
Epoch 9: Train loss 2.1669, Val loss 2.0953
Epoch 10: Train loss 2.1617, Val loss 2.0941
Epoch 11: Train loss 2.1573, Val loss 2.0918
Epoch 12: Train loss 2.1538, Val loss 2.0895
Epoch 13: Train loss 2.1505, Val loss 2.0892
Epoch 14: Train loss 2.1486, Val loss 2.0874
Epoch 15: Train loss 2.1460, Val loss 2.0858
Epoch 16: Train loss 2.1435, Val loss 2.0855
Epoch 17: Train loss 2.1417, Val loss 2.0863
Epoch 18: Train loss 2.1397, Val loss 2.0826
Epoch 19: Train loss 2.1383, Val loss 2.0815
Epoch 20: Train loss 2.1371, Val loss 2.0823
Epoc

[I 2025-03-07 12:07:36,156] Trial 14 finished with value: 2.0745204591968585 and parameters: {'batch_size': 32, 'lr': 0.00010999744095048953, 'hidden_dim': 384, 'weight_decay': 5.383174846292139e-06}. Best is trial 13 with value: 2.05181546670681.


Epoch 29: Train loss 2.1285, Val loss 2.0749
Best hyperparameters: {'batch_size': 32, 'lr': 0.0005587614855267997, 'hidden_dim': 512, 'weight_decay': 6.056437420027853e-06}
Search completed in 377.77 minutes.


In [10]:
train_losses = []
val_losses = []
global train_losses, val_losses

train_sae(layer6_embeddings, "sae_layer6_1.pth", train_losses=[],
          val_losses=val_losses, epochs=500, device=device)

/Users/mariam/opt/anaconda3/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch [1/500], Train Loss: 2.3618, Val Loss: 2.2800
Epoch [2/500], Train Loss: 2.2735, Val Loss: 2.2984
Epoch [3/500], Train Loss: 2.2630, Val Loss: 2.2754
Epoch [4/500], Train Loss: 2.2570, Val Loss: 2.2771
Epoch [5/500], Train Loss: 2.2525, Val Loss: 2.2562
Epoch [6/500], Train Loss: 2.2502, Val Loss: 2.2787
Epoch [7/500], Train Loss: 2.2485, Val Loss: 2.2574
Epoch [8/500], Train Loss: 2.2471, Val Loss: 2.2740
Epoch [9/500], Train Loss: 2.2461, Val Loss: 2.2814
Epoch [10/500], Train Loss: 2.2452, Val Loss: 2.2925
Epoch [11/500], Train Loss: 2.2444, Val Loss: 2.2744
Epoch [12/500], Train Loss: 2.2260, Val Loss: 2.2265
Epoch [13/500], Train Loss: 2.2237, Val Loss: 2.2189
Epoch [14/500], Train Loss: 2.2233, Val Loss: 2.2273
Epoch [15/500], Train Loss: 2.2230, Val Loss: 2.2203
Epoch [16/500], Train Loss: 2.2226, Val Loss: 2.2392
Epoch [17/500], Train Loss: 2.2220, Val Loss: 2.2464
Epoch [18/500], Train Loss: 2.2219, Val Loss: 2.2346
Epoch [19/500], Train Loss: 2.2212, Val Loss: 2.2189
Ep

In [11]:
train_losses = []
val_losses = []
global train_losses, val_losses

train_sae(layer12_embeddings, "sae_layer12.pth", train_losses=[],
          val_losses=val_losses, epochs=500, device=device)

Epoch [1/500], Train Loss: 5.9004, Val Loss: 5.7176
Epoch [2/500], Train Loss: 5.7059, Val Loss: 5.6882
Epoch [3/500], Train Loss: 5.6882, Val Loss: 5.7546
Epoch [4/500], Train Loss: 5.6805, Val Loss: 5.7202
Epoch [5/500], Train Loss: 5.6752, Val Loss: 5.7325
Epoch [6/500], Train Loss: 5.6718, Val Loss: 5.6864
Epoch [7/500], Train Loss: 5.6699, Val Loss: 5.6657
Epoch [8/500], Train Loss: 5.6674, Val Loss: 5.6849
Epoch [9/500], Train Loss: 5.6654, Val Loss: 5.7040
Epoch [10/500], Train Loss: 5.6632, Val Loss: 5.7012
Epoch [11/500], Train Loss: 5.6616, Val Loss: 5.7231
Epoch [12/500], Train Loss: 5.6603, Val Loss: 5.6998
Epoch [13/500], Train Loss: 5.6603, Val Loss: 5.6749
Epoch [14/500], Train Loss: 5.6296, Val Loss: 5.5976
Epoch [15/500], Train Loss: 5.6270, Val Loss: 5.6285
Epoch [16/500], Train Loss: 5.6266, Val Loss: 5.6242
Epoch [17/500], Train Loss: 5.6256, Val Loss: 5.6264
Epoch [18/500], Train Loss: 5.6253, Val Loss: 5.6167
Epoch [19/500], Train Loss: 5.6253, Val Loss: 5.5838
Ep